<h1>Pipeline Model Deployment</h1>

Once we have built and trained our models for feature engineering (using Amazon SageMaker Processing and SKLearn) and binary classification (using the XGBoost open-source container for Amazon SageMaker), we can choose to deploy them in a pipeline on Amazon SageMaker Hosting, by creating an Inference Pipeline.
https://docs.aws.amazon.com/sagemaker/latest/dg/inference-pipelines.html

This notebook demonstrates how to create a pipeline with the SKLearn model for feature engineering and the XGBoost model for binary classification.

Let's define the variables first.

In [1]:
import boto3
import sagemaker

role = sagemaker.get_execution_role()
region = boto3.Session().region_name
sagemaker_session = sagemaker.Session()
bucket_name = sagemaker_session.default_bucket()
prefix = 'endtoendmlsm'

print(region)
print(role)
print(bucket_name)

eu-west-1
arn:aws:iam::825935527263:role/service-role/AmazonSageMaker-ExecutionRole-endtoendml
sagemaker-eu-west-1-825935527263


First, we need to create two Amazon SageMaker **Model** objects, which associate the artifacts of training (serialized model artifacts in Amazon S3) to the Docker container used for inference.

In order to do that, we need to get the paths to our serialized models in Amazon S3.
<ul>
    <li>For the SKLearn model, in Step 02 (data exploration and feature engineering) we defined the path where the artifacts are saved</li>
    <li>For the XGBoost model, we need to find the path based on Amazon SageMaker's naming convention</li>
</ul>

In [2]:
from notebook_utilities import get_latest_training_job_name, get_training_job_s3_model_artifacts

# SKLearn model path.
sklearn_model_path = 's3://{0}/{1}/output/sklearn/model.tar.gz'.format(bucket_name, prefix)

training_base_job_name = 'end-to-end-ml-sm-xgb'
latest_training_job_name = get_latest_training_job_name(training_base_job_name)
# XGBoost model path.
xgboost_model_path = get_training_job_s3_model_artifacts(latest_training_job_name)

print('SKLearn model path: ' + sklearn_model_path)
print('XGBoost model path: ' + xgboost_model_path)

SKLearn model path: s3://sagemaker-eu-west-1-825935527263/endtoendmlsm/output/sklearn/model.tar.gz
XGBoost model path: s3://sagemaker-eu-west-1-825935527263/endtoendmlsm/output/end-to-end-ml-sm-xgb-2020-03-26-15-21-27-536/output/model.tar.gz


## SKLearn Featurizer Model

Let's build the SKLearn model. For hosting this model we also provide a custom inference script, that is used to process the inputs and outputs and execute the transform.

The inference script is implemented in the `sklearn_source_dir/inference.py` file.

In [8]:
!pygmentize sklearn_source_dir/inference.py

from __future__ import print_function

import sys
from io import StringIO
import os
import csv
import json
import numpy as np
import pandas as pd

from sklearn.externals import joblib

from sagemaker_containers.beta.framework import (
    content_types, encoders, env, modules, transformer, worker)

feature_columns_names = ['turbine_id', 'turbine_type', 'wind_speed', 'rpm_blade', 'oil_temperature',
                         'oil_level', 'temperature', 'humidity', 'vibrations_frequency', 'pressure', 'wind_direction']

def input_fn(input_data, content_type):
    print(input_data)
    
    if content_type == 'text/csv':
        df = pd.read_csv(StringIO(input_data), header=None)
        if len(df.columns) == len(feature_columns_names):
            df.columns = feature_columns_names 
        return df
    else:
        raise ValueError("{} not supported by script!".format(content_type))
        
def predict_fn(input_data, model):
    features = model.transform(input_data)
    return features

In [9]:
import time
from sagemaker.sklearn import SKLearnModel

code_location = 's3://{0}/{1}/code'.format(bucket_name, prefix)

sklearn_model = SKLearnModel(name='end-to-end-ml-sm-skl-model-{0}'.format(str(int(time.time()))),
                             model_data=sklearn_model_path,
                             entry_point='inference.py',
                             source_dir='sklearn_source_dir/',
                             code_location=code_location,
                             role=role,
                             sagemaker_session=sagemaker_session
                             )

## XGBoost Model

In [10]:
!pygmentize xgboost_source_dir/inference.py

import os
import pickle as pkl
import json
import numpy as np
import xgboost as xgb
import pandas as pd

from sagemaker_containers.beta.framework import (
    content_types, encoders, env, modules, transformer, worker)

from sagemaker_xgboost_container import encoder as xgb_encoders

def input_fn(input_data, content_type):
    print('Printing inputs.')
    print(content_type)
    print(input_data)
    
    if content_type == content_types.JSON:
        obj = json.loads(input_data)
        features = obj['instances'][0]['features']
        array = np.array(features).reshape((1, -1))
        return xgb.DMatrix(array)
    else:
        return xgb_encoders.decode(input_data, content_type)

def model_fn(model_dir):
    model_file = model_dir + '/model.bin'
    model = pkl.load(open(model_file, 'rb'))
    return model


In [11]:
import time
from sagemaker.xgboost import XGBoostModel

code_location = 's3://{0}/{1}/code'.format(bucket_name, prefix)

xgboost_model = XGBoostModel(name='end-to-end-ml-sm-xgb-model-{0}'.format(str(int(time.time()))),
                             model_data=xgboost_model_path,
                             entry_point='inference.py',
                             source_dir='xgboost_source_dir/',
                             code_location=code_location,
                             framework_version='0.90-2',
                             role=role, 
                             sagemaker_session=sagemaker_session
                             )

## Pipeline Model

Once we have models ready, we can deploy them in a pipeline:

In [12]:
import sagemaker
import time
from sagemaker.pipeline import PipelineModel

pipeline_model_name = 'end-to-end-ml-sm-xgb-skl-pipeline-{0}'.format(str(int(time.time())))

pipeline_model = PipelineModel(
    name=pipeline_model_name, 
    role=role,
    models=[
        sklearn_model, 
        xgboost_model],
    sagemaker_session=sagemaker_session)

endpoint_name = 'end-to-end-ml-sm-pipeline-endpoint-{0}'.format(str(int(time.time())))
print(endpoint_name)

pipeline_model.deploy(initial_instance_count=1, 
                      instance_type='ml.c5.xlarge', 
                      endpoint_name=endpoint_name)

end-to-end-ml-sm-pipeline-endpoint-1585316383
-------------!

<span style="color: red; font-weight:bold">Please take note of the endpoint name, since it will be used in the next workshop module.</span>

<h2>Getting inferences</h2>

Now we can try invoking our pipeline of models and try getting some inferences:

In [17]:
from sagemaker.predictor import json_serializer, csv_serializer, csv_deserializer, json_deserializer, RealTimePredictor
from sagemaker.content_types import CONTENT_TYPE_CSV, CONTENT_TYPE_JSON

predictor = RealTimePredictor(
    endpoint=endpoint_name,
    sagemaker_session=sagemaker_session,
    content_type=CONTENT_TYPE_CSV,
    accept=CONTENT_TYPE_JSON)

predictor.serializer = csv_serializer
predictor.deserializer = json_deserializer

payload = "TID008,HAWT,64,80,46,21,55,55,7,34,SE"
print(predictor.predict(payload))

[0.6588892936706543]


Once we have tested the endpoint, we can move to the next workshop module. Please access the module <a href="https://github.com/giuseppeporcelli/end-to-end-ml-application/tree/master/05_API_Gateway_and_Lambda" target="_blank">05_API_Gateway_and_Lambda</a> on GitHub to continue.